In [1]:
%matplotlib inline
#import malariagen_data
import allel
import malariagen_data
import numpy as np
import pandas as pd
import seaborn as sns

import sys
# adding Folder_2 to the system path
sys.path.insert(0, '/home/sanj/projects/gaardian/workflow/scripts/')
import probetools as probe

In [2]:
def load_metahaps(cohorts, names, metaquery = None):
    meta = ag3.sample_metadata(cohorts)
    meta = meta.query("sample_id in @names")
    meta['sample_id'] = pd.Categorical(meta['sample_id'], categories = names, ordered = True)
    if metaquery != None: meta = meta.query(metaquery)
    meta = meta.sort_values(by='sample_id')
    metahaps = pd.DataFrame(np.repeat(meta.values, 2, axis=0))
    metahaps.columns = meta.columns
    return(metahaps)

### Preparing the haplotype data for fineStructure

In [3]:
dataset = 'gaardian'
contig='3L'
gaardian_sample_set = ['1244-VO-GH-YAWSON-VMF00149']

ag3 = malariagen_data.Ag3(pre=True)

In [23]:
haps = ag3.haplotypes(region=contig, sample_sets=gaardian_sample_set, analysis='gamb_colu')
sp_ids = haps['sample_id'].compute().values

In [25]:
haps['variant_position'].shape

(11354020,)

In [28]:
np.unique(haps['variant_position']).shape

(11354020,)

In [5]:
meta = ag3.sample_metadata(gaardian_sample_set)
meta = meta.query("sample_id in @sp_ids")
meta['sample_id'] = pd.Categorical(meta['sample_id'], categories = sp_ids, ordered = True)
meta = meta.sort_values(by='sample_id')

samples = meta[['partner_sample_id', 'location']].copy()
samples.loc[:, 'location'] = samples.loc[:, 'location'].str.split(".").str.get(0)
samples.loc[:, 'keep'] = 1

print(samples.shape)
samples.head(3)

(485, 3)


,partner_sample_id,location,keep
0,WA-2001,Adumanu,1
1,WA-2002,Adumanu,1
2,WA-2003,Kente,1


In [103]:
samples.to_csv("../../resources/phaseHaplotypes/sample.ids", sep=" ", index=None, header=False)

### Haplotypes in phase format

In [20]:
!mkdir -pv ../../resources/phaseHaplotypes

mkdir: created directory '../../resources/phaseHaplotypes'


In [41]:
selection = np.random.choice(haps['variant_position'].shape[0], size=1_000_000, replace=False)
selection = np.sort(selection)

In [44]:
haps = haps.isel(variants=selection)
pos = haps['variant_position'].values

In [46]:
with open(f"../../resources/phaseHaplotypes/{dataset}_{contig}.phase", 'w') as outfile:
    outfile.write(str(samples.shape[0]*2))
    outfile.write("\n")
    outfile.write(str(haps['call_genotype'].shape[0]))
    outfile.write("\n")
    outfile.write("P ")
    for p in pos:
        outfile.write(f"{p} ")
    outfile.write("\n")
    for i in range(haps['call_genotype'].shape[1]):
        for allele in [0,1]:
            outfile.write(''.join(haps['call_genotype'][:, i, allele].values.astype(str)))
            outfile.write("\n")

In [47]:
with open(f"../../resources/phaseHaplotypes/{dataset}_{contig}.recomb", "w") as outfile:
    outfile.write("snp_pos\trecomb_per_bp\n")
    for p in pos:
        outfile.write(f"{p}\t0.0000000001")
        outfile.write("\n")